In [1]:
!pip install simpletransformers

In [31]:
import pandas as pd
import re
import numpy as np
import sklearn

In [41]:
train = pd.read_csv('../../../src/data/schemafiltereddata/TrainTestTables/CSV/small_tables_seq.csv')
test = pd.read_csv('../../../src/data/schemafiltereddata/TrainTestTables/test_tables_seq_9000.csv')

In [42]:
train["label_complete"] = train["name"] + train["class"]
test["label_complete"] = test["name"] + test["class"]

In [43]:
train.shape[0]

13236

In [48]:
test.label_complete.unique() - train.label_complete.unique()

ValueError: operands could not be broadcast together with shapes (202,) (201,) 

In [69]:

list(set(list(test.label_complete.unique())) - set(list(train.label_complete.unique())))

['mainentityofpagePerson']

In [68]:
len(test.label_complete.unique())

202

In [73]:
test = test[test['label_complete']!='mainentityofpagePerson']


In [56]:
train.drop(train[(train['name']=='inalbum') &(train['class']=='MusicRecording')].index, inplace=True)
train.drop(train[(train['name']=='performer') &(train['class']=='Event')].index, inplace=True)
train.drop(train[(train['name']=='eventattendancemode') &(train['class']=='Event')].index, inplace=True)
train.drop(train[(train['name']=='eventstatus') &(train['class']=='Event')].index, inplace=True)

In [59]:
test.drop(test[(test['name']=='inalbum') &(test['class']=='MusicRecording')].index, inplace=True)
test.drop(test[(test['name']=='performer') &(test['class']=='Event')].index, inplace=True)
test.drop(test[(test['name']=='eventattendancemode') &(test['class']=='Event')].index, inplace=True)
test.drop(test[(test['name']=='eventstatus') &(test['class']=='Event')].index, inplace=True)
test.drop(test[(test['name']=='mainentityofpagePerson') ].index, inplace=True)
test.shape

(48535, 5)

In [ ]:
test.drop(test[(test['name']=='eventstatus') &(test['class']=='Event')].index, inplace=True)

In [60]:
test.head()

,Unnamed: 0,text,name,class,label_complete
0,0,Plauen | Indonesien Freiburg | Iraq-Kurdistan ...,name,Event,nameEvent
1,1,2022-03-27T16:30:00+01:00 2020-11-15T22:00:00+...,enddate,Event,enddateEvent
2,2,"{'name': 'Abenteuer Arena'},{'name': 'Paulussa...",location,Event,locationEvent
3,3,"{'name': 'Freiträumer Festival'},{'name': 'MUN...",organizer,Event,organizerEvent
4,4,Máy làm kem Sunny Way RB1119A TỦ MÁT 2 CÁNH KÍ...,name,Product,nameProduct


In [61]:
train.drop(['Unnamed: 0', 'class'], axis=1, inplace=True)
test.drop(['Unnamed: 0', 'class'], axis=1, inplace=True)

In [62]:
train['text'] = train.text.apply(lambda x: re.sub("[^0-9a-zA-Z-@]+", " ", x))
test['text'] = test.text.apply(lambda x: re.sub("[^0-9a-zA-Z-@]+", " ", x))

In [74]:
len(set(test.label_complete.unique()))

201

In [75]:
len(set(train.label_complete.unique()))

201

In [76]:
num_columns = len(set.union(set(test.label_complete.unique()), set(train.label_complete.unique())))

In [77]:
set(test.label_complete.unique()).difference(set(train.label_complete.unique()))

set()

In [80]:
labels_new = {'worstratingProduct': 0,
 'acceptsreservationsRestaurant': 1,
 'additionalnamePerson': 2,
 'additionalpropertyProduct': 3,
 'addressHotel': 4,
 'addressLocalBusiness': 5,
 'addressPerson': 6,
 'addressProduct': 7,
 'addressRestaurant': 8,
 'addresslocalityLocalBusiness': 9,
 'addressregionLocalBusiness': 10,
 'affiliationPerson': 11,
 'aggregateratingBook': 12,
 'aggregateratingCreativeWork': 13,
 'aggregateratingHotel': 14,
 'aggregateratingLocalBusiness': 15,
 'aggregateratingProduct': 16,
 'aggregateratingRecipe': 17,
 'aggregateratingRestaurant': 18,
 'alternatenameProduct': 19,
 'alternativeheadlineCreativeWork': 20,
 'areaservedLocalBusiness': 21,
 'articlebodyCreativeWork': 22,
 'audienceProduct': 23,
 'authorBook': 24,
 'authorCreativeWork': 25,
 'authorProduct': 26,
 'authorRecipe': 27,
 'availabilityProduct': 28,
 'bestratingProduct': 29,
 'birthdatePerson': 30,
 'birthplacePerson': 31,
 'bookeditionBook': 32,
 'bookformatBook': 33,
 'brandProduct': 34,
 'breadcrumbProduct': 35,
 'byartistMusicAlbum': 36,
 'byartistMusicRecording': 37,
 'categoryProduct': 38,
 'citystatezipLocalBusiness': 39,
 'colorProduct': 40,
 'commentcountCreativeWork': 41,
 'conditionProduct': 42,
 'contactpointLocalBusiness': 43,
 'contactpointPerson': 44,
 'cookingmethodRecipe': 45,
 'cooktimeRecipe': 46,
 'copyrightholderCreativeWork': 47,
 'copyrightyearCreativeWork': 48,
 'creatorCreativeWork': 49,
 'datecreatedCreativeWork': 50,
 'datecreatedMusicRecording': 51,
 'datemodifiedCreativeWork': 52,
 'datemodifiedRecipe': 53,
 'datepublishedBook': 54,
 'datepublishedCreativeWork': 55,
 'datepublishedMusicRecording': 56,
 'datepublishedProduct': 57,
 'datepublishedRecipe': 58,
 'deathdatePerson': 59,
 'depthProduct': 60,
 'disambiguatingdescriptionProduct': 61,
 'doortimeEvent': 62,
 'durationEvent': 63,
 'durationMusicRecording': 64,
 'emailHotel': 65,
 'emailLocalBusiness': 66,
 'emailPerson': 67,
 'enddateEvent': 68,
 'episodenumberTVEpisode': 69,
 'worksforPerson': 70,
 'familynamePerson': 71,
 'faxnumberLocalBusiness': 72,
 'faxnumberPerson': 73,
 'genderPerson': 74,
 'genreBook': 75,
 'genreCreativeWork': 76,
 'genreMusicRecording': 77,
 'geoHotel': 78,
 'geoLocalBusiness': 79,
 'geoPlace': 80,
 'geoProduct': 81,
 'geoRestaurant': 82,
 'givennamePerson': 83,
 'gtin12Product': 84,
 'gtin13Product': 85,
 'gtin14Product': 86,
 'gtin8Product': 87,
 'gtinProduct': 88,
 'hasmapLocalBusiness': 89,
 'hasmenuRestaurant': 90,
 'headlineCreativeWork': 91,
 'headlineRecipe': 92,
 'heightPerson': 93,
 'heightProduct': 94,
 'homelocationPerson': 95,
 'identifierProduct': 96,
 'worklocationPerson': 97,
 'ingredientsRecipe': 98,
 'inlanguageBook': 99,
 'inlanguageCreativeWork': 100,
 'interactioncountCreativeWork': 101,
 'interactionstatisticCreativeWork': 102,
 'interactiontypeCreativeWork': 103,
 'isbnBook': 104,
 'ispartofRecipe': 105,
 'isrelatedtoProduct': 106,
 'issimilartoProduct': 107,
 'itemconditionProduct': 108,
 'itemlistelementProduct': 109,
 'jobtitlePerson': 110,
 'keywordsCreativeWork': 111,
 'keywordsRecipe': 112,
 'knowslanguagePerson': 113,
 'legalnameLocalBusiness': 114,
 'locationEvent': 115,
 'locationLocalBusiness': 116,
 'mainentityofpageCreativeWork': 117,
 'founderLocalBusiness': 118,
 'mainentityofpageProduct': 119,
 'mainentityofpageRecipe': 120,
 'makesofferPerson': 121,
 'manufacturerProduct': 122,
 'materialProduct': 123,
 'memberofPerson': 124,
 'menuRestaurant': 125,
 'modelProduct': 126,
 'mpnProduct': 127,
 'nameBook': 128,
 'nameCreativeWork': 129,
 'nameEvent': 130,
 'nameHotel': 131,
 'nameLocalBusiness': 132,
 'nameMusicAlbum': 133,
 'nameMusicRecording': 134,
 'namePlace': 135,
 'nameProduct': 136,
 'nameRecipe': 137,
 'nameRestaurant': 138,
 'nameTVEpisode': 139,
 'nationalityPerson': 140,
 'numberofpagesBook': 141,
 'numtracksMusicAlbum': 142,
 'nutritionRecipe': 143,
 'weightProduct': 144,
 'offersBook': 145,
 'offersCreativeWork': 146,
 'offersProduct': 147,
 'openinghoursLocalBusiness': 148,
 'openinghoursRestaurant': 149,
 'openinghoursspecificationLocalBusiness': 150,
 'openinghoursspecificationPlace': 151,
 'openinghoursspecificationRestaurant': 152,
 'organizerEvent': 153,
 'partofseriesTVEpisode': 154,
 'paymentacceptedLocalBusiness': 155,
 'widthProduct': 156,
 'performersEvent': 157,
 'performtimeRecipe': 158,
 'postalcodeLocalBusiness': 159,
 'preptimeRecipe': 160,
 'priceProduct': 161,
 'pricecurrencyProduct': 162,
 'pricerangeHotel': 163,
 'pricerangeLocalBusiness': 164,
 'pricerangeRestaurant': 165,
 'publisherBook': 166,
 'publisherCreativeWork': 167,
 'publisherRecipe': 168,
 'ratingvalueProduct': 169,
 'recipecategoryRecipe': 170,
 'recipecuisineRecipe': 171,
 'recipeingredientRecipe': 172,
 'recipeinstructionsRecipe': 173,
 'recipeyieldRecipe': 174,
 'releasedateProduct': 175,
 'reviewLocalBusiness': 176,
 'reviewProduct': 177,
 'reviewRecipe': 178,
 'reviewcountProduct': 179,
 'reviewsProduct': 180,
 'sameasLocalBusiness': 181,
 'sameasPerson': 182,
 'sameasPlace': 183,
 'servescuisineRestaurant': 184,
 'shop-currencyProduct': 185,
 'starratingHotel': 186,
 'streetaddressLocalBusiness': 187,
 'suitablefordietRecipe': 188,
 'telephoneHotel': 189,
 'telephoneLocalBusiness': 190,
 'telephonePerson': 191,
 'telephonePlace': 192,
 'telephoneRestaurant': 193,
 'titleProduct': 194,
 'totaltimeRecipe': 195,
 'trackMusicAlbum': 196,
 'typicalagerangeEvent': 197,
 'versionCreativeWork': 198,
 'weightPerson': 199,
 'offerdetailsProduct': 200
 #'eventattendancemodeEvent': 201,
 #'performerEvent': 202,
 #'inalbumMusicRecording': 203,
 #'eventstatusEvent': 204,
 #'mainentityofpagePerson': 118,
 }

In [81]:
train['label'] = train.label_complete.map(labels_new)
test['label'] = test.label_complete.map(labels_new)
train.drop(['label_complete', 'name'], axis=1, inplace=True)
test.drop(['label_complete', 'name'], axis=1, inplace=True)

/tmp/ipykernel_1174321/2948527013.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['label'] = test.label_complete.map(labels_new)
/home/bizer-tp2021/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [82]:
len(train.label.unique())

201

In [83]:
len(test.label.unique())

201

In [84]:
set(test.label.unique()).difference(set(train.label.unique()))

set()

In [85]:
train.to_csv('Data/small_train_prepped.csv', index=False)

In [86]:
test.to_csv('Data/test_prepped.csv', index=False)

In [87]:
# arguments
train_args = {
    'num_train_epochs': 25,
    'overwrite_output_dir': True
}

In [90]:
from simpletransformers.classification import ClassificationModel

model = ClassificationModel(
    "huawei-noah/TinyBERT_General_4L_312D", 'huawei-noah/TinyBERT_General_4L_312D', args=train_args, num_labels=201
)

KeyError: 'huawei-noah/TinyBERT_General_4L_312D'